In [ ]:
import cv2
import mediapipe as mp
from collections import deque
import numpy as np
from deepface import DeepFace

# ---------------- Pose Setup ----------------
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

history = deque(maxlen=5)

def get_posture_score(landmarks):
    left_shoulder = landmarks[11]
    right_shoulder = landmarks[12]
    left_hip = landmarks[23]
    right_hip = landmarks[24]

    shoulder_y = (left_shoulder.y + right_shoulder.y) / 2
    hip_y = (left_hip.y + right_hip.y) / 2

    upright_ratio = max(0, min(1, 1 - abs(shoulder_y - hip_y)))
    return upright_ratio * 100

def torso_movement(landmarks, prev_landmarks):
    if prev_landmarks is None:
        return 0
    keypoints = [11, 12, 23, 24]  # shoulders and hips
    movement = 0
    for kp in keypoints:
        dx = landmarks[kp].x - prev_landmarks[kp].x
        dy = landmarks[kp].y - prev_landmarks[kp].y
        movement += np.sqrt(dx**2 + dy**2)
    return movement / len(keypoints)

prev_landmarks = None
movement_threshold = 0.005

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_frame)

    # ---------------- Pose & Engagement ----------------
    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        posture_score = get_posture_score(landmarks)
        move_score = torso_movement(landmarks, prev_landmarks)

        history.append(posture_score)
        avg_posture = sum(history) / len(history)

        if move_score < movement_threshold:
            engagement_text = "No Engagement"
        else:
            engagement_value = max(0, min(100, avg_posture + 10))
            engagement_text = f"Engagement: {engagement_value:.1f}%"

        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        cv2.putText(frame, engagement_text, (30, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        prev_landmarks = landmarks
    else:
        cv2.putText(frame, "No person detected",
                    (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        prev_landmarks = None

    # ---------------- Emotion Detection (DeepFace) ----------------
    try:
        analysis = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False)
        dominant_emotion = analysis[0]['dominant_emotion']
        cv2.putText(frame, f"Emotion: {dominant_emotion}", (30, 100),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    except Exception as e:
        cv2.putText(frame, "Emotion: Not detected", (30, 100),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # ---------------- Display ----------------
    cv2.imshow("Engagement + Emotion Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


I0000 00:00:1755692138.029883   18500 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1755692138.033934   25275 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.2.8-1ubuntu1~24.04.1), renderer: Mesa Intel(R) UHD Graphics (TGL GT1)
W0000 00:00:1755692138.162936   25267 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1755692138.238454   25271 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [12]:
import numpy
import torch
import transformers

print(numpy.__version__)
print(torch.__version__)
print(transformers.__version__)



2.1.3
2.8.0+cu128
4.55.2
